<a href="https://colab.research.google.com/github/jitender2622/Chat-With-PDF-Gemini/blob/main/chat_with_pdf_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 1. INSTALLATION
# ==========================================
# Installs necessary libraries for RAG, Vector Search, and PDF processing.
# Note: Specific versions are pinned to ensure compatibility.
# ==========================================

!pip install -q -U langchain==0.3.0 \
langchain-community==0.3.0 \
langchain-core==0.3.0 \
langchain-google-genai \
langchain-huggingface \
langchain-text-splitters \
faiss-cpu \
pypdf \
sentence-transformers

print("✅ Installation Complete.")
print("⚠️ NOW: Go to 'Runtime' -> 'Restart Session' (or Restart Runtime).")
print("⚠️ Do NOT run this cell again after restarting.")

In [ ]:
# ==========================================
# 2. SETUP API & PATHS
# ==========================================
# Handles secure API key input and verifies that the target PDF exists.
# ==========================================

import os
from getpass import getpass

# Define the path to the source PDF
pdf_filename = "/content/terms.pdf"  # <--- MAKE SURE THIS MATCHES YOUR FILE IN FILES TAB

# Verify file existence to prevent downstream errors
if not os.path.exists(pdf_filename):
    raise FileNotFoundError(f"❌ STOP: Please upload '{pdf_filename}' to the Files tab on the left!")

# Securely input Google API Key if not already set
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Paste your Google Gemini API Key here: ")

In [ ]:
# ==========================================
# 3. LIBRARY IMPORTS
# ==========================================
# Importing LangChain components for document loading, splitting,
# vector storage, and model interaction.
# ==========================================

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

print("\n🚀 Libraries imported successfully!")

In [ ]:
# ==========================================
# 4. LOAD & SPLIT PDF
# ==========================================
# Loads the raw PDF text and splits it into chunks (1000 chars).
# Overlap ensures context is preserved between chunks.
# ==========================================

print("📄 Loading and splitting PDF...")

loader = PyPDFLoader(pdf_filename)
pages = loader.load()

# Initialize splitter with overlap to maintain semantic continuity
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

print(f"✅ Split into {len(splits)} chunks.")

In [ ]:
# ==========================================
# 5. EMBEDDINGS (LOCAL)
# ==========================================
# Converts text chunks into vector embeddings using a local HuggingFace model.
# These vectors are stored in a FAISS index for efficient similarity search.
# ==========================================

print("🧠 Creating Local Embeddings (HuggingFace)...")

# Using 'all-MiniLM-L6-v2' (Local CPU model) to avoid API rate limits
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the vector store
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# Configure the retriever to fetch the top 5 most relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

print("✅ Vector Database Ready.")

In [ ]:
# ==========================================
# 6. BUILD RAG CHAIN
# ==========================================
# Initializes the Gemini LLM and constructs the retrieval chain.
# This links the Vector Store (Context) with the LLM (Reasoning).
# ==========================================

print("🔗 Building the RAG Chain...")

# Initialize Gemini 2.0 Flash with low temperature for factual consistency
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)

# Define the system instructions
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Assemble the processing chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("✅ RAG Chain Assembled.")

In [ ]:
# ==========================================
# 7. EXECUTION
# ==========================================
# Runs a test query against the RAG chain.
# ==========================================

print("\n🤖 SYSTEM READY! Testing with a question...")

# Define your query here
query = "What is Transformers?"
print(f"❓ Question: {query}")

# Invoke the chain
response = rag_chain.invoke({"input": query})

# Output the result
print("\n💡 Answer:")
print(response["answer"])